In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [12,8]
plt.rcParams["font.size"] = 14
import warnings
warnings.simplefilter("ignore")
import datetime as dt
from scipy import io
from joblib import load
from grafana_utils import *

user_node_memory = load_grafana_data("../data/actual_memory_use_user_nodes.csv")
core_node_memory = load_grafana_data("../data/actual_memory_use_core_nodes.csv")
active_users = load_grafana_data("../data/active_users_by_hub.csv")
non_running_user_pods = load_grafana_data("../data/non_running_user_pods.csv")
user_pool_cluster_utilization = load_grafana_data("../data/user_pool_cluster_utilization.csv")
current_users = load_grafana_data("../data/hub_current_users.csv")
response_codes = load_grafana_data("../data/hub_response_codes.csv")

In [34]:
user_node_memory["timestamp"] = user_node_memory["timestamp"].apply(pd.to_datetime)
core_node_memory["timestamp"] = core_node_memory["timestamp"].apply(pd.to_datetime)
active_users["timestamp"] = active_users["timestamp"].apply(pd.to_datetime)
non_running_user_pods["timestamp"] = non_running_user_pods["timestamp"].apply(pd.to_datetime)
user_pool_cluster_utilization["timestamp"] = user_pool_cluster_utilization["timestamp"].apply(pd.to_datetime)
current_users["timestamp"] = current_users["timestamp"].apply(pd.to_datetime)
response_codes["timestamp"] = response_codes["timestamp"].apply(pd.to_datetime)

In [35]:
user_node_memory = user_node_memory.groupby("timestamp").sum().reset_index()[["timestamp", "value"]]
core_node_memory = core_node_memory.groupby("timestamp").sum().reset_index()[["timestamp", "value"]]
active_users = active_users.groupby("timestamp").sum().reset_index()[["timestamp", "value"]]
non_running_user_pods = non_running_user_pods.groupby("timestamp").sum().reset_index()[["timestamp", "value"]]
user_pool_cluster_utilization = (user_pool_cluster_utilization
                                 .groupby("timestamp").sum().reset_index()[["timestamp", "value"]])
current_users = current_users.groupby("timestamp").sum().reset_index()[["timestamp", "value"]]
response_codes = (response_codes[response_codes["series"].isin(["500", "503"])]
                  .groupby("timestamp").sum().reset_index()[["timestamp", "value"]])

In [41]:
sub_merge_1 = pd.merge(
    user_node_memory,
    core_node_memory,
    on="timestamp",
    how="outer"
).rename({
    "value_x": "user_node_memory",
    "value_y": "core_node_memory"
}, axis=1)

sub_merge_2 = pd.merge(
    active_users,
    non_running_user_pods,
    on="timestamp",
    how="outer"
).rename({
    "value_x": "active_users",
    "value_y": "non_running_user_pods"
}, axis=1)

sub_merge_3 = pd.merge(
    user_pool_cluster_utilization,
    current_users,
    on="timestamp",
    how="outer"
).rename({
    "value_x": "user_pool_cluster_utilization",
    "value_y": "current_users"
}, axis=1)


sub_merge_4 = pd.merge(
    sub_merge_1,
    sub_merge_2,
    on="timestamp",
    how="outer"
)

sub_merge_5 = pd.merge(
    sub_merge_3,
    sub_merge_4,
    on="timestamp",
    how="outer"
)

merged = pd.merge(
    sub_merge_5,
    response_codes,
    on="timestamp",
    how="outer"
).rename({
    "value": "500_503_responses"
}, axis=1)

In [43]:
merged = merged.fillna(0)
merged

,timestamp,user_pool_cluster_utilization,current_users,user_node_memory,core_node_memory,active_users,non_running_user_pods,500_503_responses
0,2019-09-18 19:00:00,0.952093,639.0,1.882692e+11,8.594047e+09,725.0,0.0,495.000000
1,2019-09-19 07:00:00,0.723973,644.0,1.891179e+11,8.434262e+09,736.0,1.0,263.750000
2,2019-09-19 19:00:00,0.717522,451.0,1.544145e+11,8.457826e+09,598.0,0.0,230.000000
3,2019-09-20 07:00:00,0.523064,318.0,1.189684e+11,8.480592e+09,423.0,0.0,201.250000
4,2019-09-20 19:00:00,0.838836,423.0,1.388345e+11,8.544907e+09,561.0,0.0,453.750000
5,2019-09-21 07:00:00,0.445868,199.0,9.471130e+10,7.551185e+09,270.0,1.0,206.250000
6,2019-09-21 19:00:00,0.412108,140.0,9.737378e+10,7.608734e+09,242.0,0.0,410.000000
7,2019-09-22 07:00:00,0.724085,230.0,1.250565e+11,8.191382e+09,406.0,0.0,303.750000
8,2019-09-22 19:00:00,0.633029,191.0,1.097639e+11,8.324366e+09,349.0,0.0,240.000000
9,2019-09-23 07:00:00,0.914458,413.0,1.658575e+11,8.547353e+09,597.0,0.0,331.250000


In [44]:
merged.sort_values("timestamp")

,timestamp,user_pool_cluster_utilization,current_users,user_node_memory,core_node_memory,active_users,non_running_user_pods,500_503_responses
361,2019-09-11 19:00:00,0.000000,604.0,0.000000e+00,0.000000e+00,0.0,0.0,501.250000
362,2019-09-12 07:00:00,0.000000,504.0,0.000000e+00,0.000000e+00,0.0,0.0,108.750000
363,2019-09-12 19:00:00,0.000000,391.0,0.000000e+00,0.000000e+00,0.0,0.0,308.750000
364,2019-09-13 07:00:00,0.000000,259.0,0.000000e+00,0.000000e+00,0.0,0.0,302.500000
365,2019-09-13 19:00:00,0.000000,406.0,0.000000e+00,0.000000e+00,0.0,0.0,125.000000
366,2019-09-14 07:00:00,0.000000,105.0,0.000000e+00,0.000000e+00,0.0,0.0,218.750000
367,2019-09-14 19:00:00,0.000000,115.0,0.000000e+00,0.000000e+00,0.0,0.0,414.998271
368,2019-09-15 07:00:00,0.000000,155.0,0.000000e+00,0.000000e+00,0.0,0.0,400.001667
369,2019-09-15 19:00:00,0.000000,109.0,0.000000e+00,0.000000e+00,0.0,0.0,175.000000
370,2019-09-16 07:00:00,0.000000,217.0,0.000000e+00,0.000000e+00,0.0,0.0,217.500000


assume missing hours are 0

In [55]:
hours_delta = (dt.datetime(2020, 3, 17) - dt.datetime(2019, 9, 11)).days * 24
for i in range(hours_delta):
    time = dt.datetime(2019, 9, 11) + dt.timedelta(hours=i)
    if time not in merged["timestamp"]:
        values = {l : 0 for l in merged.columns[1:]}
        values["timestamp"] = time
        merged = merged.append(values, ignore_index=True)

In [62]:
merged = merged.groupby("timestamp").sum().reset_index() # can do since added rows are all 0

In [63]:
hours_delta == merged.shape[0]

True

In [64]:
merged.to_csv("../data/merged_grafana.csv", index=False)